In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_machine_learning.circuit.library import QNNCircuit
from qiskit_machine_learning.connectors import TorchConnector
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit import QuantumCircuit

In [ ]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_provider import IBMProvider
from qiskit_ibm_runtime import Estimator,Options

In [1]:
import time
import matplotlib.pyplot as plt

In [ ]:
# Prepare the dataset
df_train = pd.read_csv("db_sc1_bluetooth.csv")
df_test = pd.read_csv("Tests_Scenario1_bluetooth.csv")

X_train = df_train[["RSSI A", "RSSI B", "RSSI C"]].values  # Use all three features
Y_train = df_train[["x", "y"]].values

X_test = df_test[["RSSI A", "RSSI B", "RSSI C"]].values
Y_test = df_test[["x", "y"]].values

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32)

# Create a DataLoader for training
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

In [ ]:
provider = IBMProvider('Your API token')
qcomp = provider.get_backend('backend name')
options = Options(optimization_level=1)
pass_manager = generate_preset_pass_manager(optimization_level=1)
estimator = Estimator(backend='backend name',options=options)

In [ ]:
# Define the Quantum Neural Network (QNN)
feature_map = ZZFeatureMap(3) # can define the feature_map using any other from Qiskit or can be custom-defined
ansatz = RealAmplitudes(3, reps=3) # most commonly used ansatz, any custom ansatz can be used  
qc = QuantumCircuit(3)
qc.compose(feature_map, inplace=True)
qc.compose(ansatz, inplace=True)

In [ ]:
qnn = EstimatorQNN(
    circuit=qc,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    input_gradients=True
)

In [ ]:
# Define the classical neural network
class ClassicalNN(nn.Module):
    def __init__(self):
        super(ClassicalNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.fc2 = nn.Linear(32, 2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Define the complete model
class HybridModel(nn.Module):
    def __init__(self, qnn, classical_nn):
        super(HybridModel, self).__init__()
        self.qnn = TorchConnector(qnn)
        self.classical_nn = classical_nn

    def forward(self, x):
        x_qnn = self.qnn(x)
        x_classical = self.classical_nn(x)
        return x_qnn + x_classical

In [ ]:
# Initialize the models and optimizer
classical_nn = ClassicalNN()
hybrid_model = HybridModel(qnn, classical_nn)
optimizer = optim.Adam(hybrid_model.parameters(), lr=0.1)

In [ ]:
# Training loop
epochs = 80
training_times = []
loss_func = nn.MSELoss()
for epoch in range(epochs):
    start_time = time.time()  # Start time for the current epoch
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = hybrid_model(inputs)
        loss = loss_func(outputs, targets)
        loss.backward()
        optimizer.step()
    end_time = time.time()  # End time for the current epoch
    training_time = end_time - start_time
    training_times.append(training_time)

print(np.sum(training_times))

In [ ]:
# Plot the training times
plt.figure(figsize=(10, 5))
plt.plot(range(1, epochs + 1), training_times, marker='o')
plt.xlabel('Epoch')
plt.ylabel('Training Time (s)')
plt.title('Training Time per Epoch')
plt.grid(True)
plt.show()

In [ ]:
def create_qnn():
    feature_map = ZZFeatureMap(3)
    ansatz = RealAmplitudes(3, reps=3)
    qc = QuantumCircuit(3)
    qc.compose(feature_map, inplace=True)
    qc.compose(ansatz, inplace=True)
    # pm = generate_preset_pass_manager(backend=qcomp, optimization_level=1)
    # isa_circuit = pm.run(qc)

    qnn = EstimatorQNN(
        circuit=qc,
        input_params=feature_map.parameters,
        weight_params=ansatz.parameters,
        input_gradients=True,
        # estimator = estimator
    )
    return qnn

In [ ]:
# Save the trained model
torch.save(hybrid_model.state_dict(), "hybrid_model_trained.pt")
qnn5 = create_qnn()
# Load the saved model
loaded_model = HybridModel(qnn5,classical_nn)
loaded_model.load_state_dict(torch.load("hybrid_model_trained.pt"))

In [ ]:
# Test the loaded model on the test data
loaded_model.eval()
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
Y_pred = loaded_model(X_test_tensor).detach().numpy()

In [ ]:
# Calculate Mean Squared Error (MSE) for testing data
mse = np.mean(np.square(Y_pred - Y_test))
print(f"Mean Squared Error (MSE) on testing data: {mse}")